# Mixture of Linear Regressions

In this note, we will solve the mixture of linear regressions problem using the moment method.

This block contains some $\LaTeX$ macros.
$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [2]:
import util
from util import *
from models import LinearRegressionsMixture
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
import sympy as sp
sp.init_printing()

Let $w_1, ..., w_K \in \Re^D$ be a set of $K$ regression coefficients. Let $x_1, ..., x_N \in \Re^D$ be a given set of data points and let $y_n = \sum_{k} \delta_k w_k^\top x_n + \epsilon$ be observed responses. Our objective is to recover $(w_k)$.

## Toy Example

Let's construct a toy example for the rest of this document.

In [3]:
K, D = 2, 2
np.random.seed(0)
pis = np.array([0.4, 0.6])
ws = np.array([[0.75, 0.25], [0.4, 0.9]])
model = LinearRegressionsMixture.generate("tmp.dat", K, D, betas = ws, weights = pis)
print("True parameters:")
print(model.weights)
print(model.betas)

True parameters:
[ 0.4  0.6]
[[ 0.75  0.25]
 [ 0.4   0.9 ]]


# Noiseless, Infinite Data Setting

In this scenario, assume that $\epsilon = 0$ and that we observe expected moments of the data. Consider the moments:
\begin{align}
\E[x^\alpha y^b] 
&= \sum_k \pi_k \E[x^\alpha y^b | \delta_k = 1] \\
&= \sum_k \pi_k \E[x^\alpha [w_k^\top x]^b] \\
&= \sum_k \pi_k \E[x^\alpha \sum_{\beta \in p(b)} w_k^\beta x^\beta] \\
&= \sum_{\beta \in p(b)} \E_\pi[w^\beta] \E[x^{\alpha + \beta}],
\end{align}
where $p(b)$ are the $d$-partitions of $b$. Note that $\E[x^{\alpha + \beta}]$ are observable quantities, and thus this simply represents a mixture over the polynomials with terms $w_k^\beta$.

In [4]:
def evaluate_mixture(ws, pis, beta):
    """
    Compute E_\pi[w^beta]
    """
    return sum(pi * util.monomial(w, beta) for w, pi in zip(ws.T, pis))
    
def compute_exact_y_moments(ws, pis, moments_x, alpha, b):
    """
    Compute the exact moments E[x^a y^b] using coefficients ws, pis and moments_x.
    """
    D, K = ws.shape
    ret = 0.
    for beta in partitions(D, b):
        ret += evaluate_mixture(ws, pis, beta) * moments_x[tuple_add(alpha, beta)]
    return ret

def describe_moment_polynomial(R, moments_x, moment_y, alpha, b):
    """
    Computes the moment polynomial for E[x^alpha, y^b]
    """
    D = len(R.symbols)
    w = R.symbols
    expr = -moment_y
    for beta in partitions(D, b):
        expr += util.monomial(w, beta) * moments_x[tuple_add(alpha, beta)]
    return expr

In [5]:
# Example 
def double_factorial(n): 
    return reduce(mul, xrange(n, 0, -2)) if n > 0 else 1
def gaussian_moments(sigma, d):
    """
    E[x^d] where x is standard gaussian with sigma
    """
    if d == 0: return 1
    elif d % 2 == 0: return double_factorial(d-1) * sigma**d
    else: return 0
def expected_gaussian_moments(sigma, alphas):
    return {alpha : prod(gaussian_moments(sigma, a) for a in alpha) for alpha in alphas}
def expected_uniform_moments(alphas):
    return {alpha : 1.}
def expected_moments_y(ws, pis, moments_x, alphabs):
    return {(alpha, b) : compute_exact_y_moments(ws, pis, moments_x, alpha, b) for alpha, b in alphabs}

In [6]:
R, _ = sp.xring(['w%d'%d for d in xrange(D)], sp.RR, sp.grevlex)

deg_b, deg_x = 3, 3
sigma = 1.0
alphas = list(dominated_elements((deg_x for _ in xrange(D))))
alphabs = [(alpha, b) for alpha in alphas for b in xrange(1,deg_b+1)]
alphas_ = list(dominated_elements((deg_x + deg_b for _ in xrange(D))))
moments_x = expected_gaussian_moments(sigma, alphas_)
moments_y = expected_moments_y(ws, pis, moments_x, alphabs)
#display(moments)

In [7]:
def get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b):
    constrs = []
    for b in xrange(1, deg_b+1):
        for alpha in util.dominated_elements((deg_x for _ in xrange(D))):
            constrs.append( describe_moment_polynomial(R, moments_x, moments_y[(alpha, b)], alpha, b) )
    return constrs

With this machinery, we can compute the moment polynomials required for the moment method magic!

In [9]:
import MomentMatrix as mm; reload(mm)

constrs = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
display(constrs)
M, sol = mm.solve_moments_with_constraints(R.symbols, constrs, 2)
M.pretty_print(sol)
print M.extract_solutions_lasserre(sol['x'], Kmax=2)


     pcost       dcost       gap    pres   dres   k/t
 0:  1.8102e+00  1.8102e+00  1e+01  3e+00  1e+00  1e+00
 1:  2.1771e+00  2.3210e+00  6e-01  3e-01  8e-02  2e-01
 2:  2.3472e+00  2.4109e+00  3e-01  1e-01  3e-02  1e-01
 3:  2.3897e+00  2.4019e+00  5e-02  2e-02  7e-03  2e-02
 4:  2.4001e+00  2.4071e+00  4e-02  2e-02  5e-03  1e-02
 5:  2.4055e+00  2.4066e+00  9e-03  4e-03  1e-03  2e-03
 6:  2.4082e+00  2.4083e+00  2e-03  1e-03  3e-04  4e-04
 7:  2.4086e+00  2.4086e+00  3e-04  1e-04  4e-05  6e-05
 8:  2.4087e+00  2.4087e+00  4e-05  2e-05  6e-06  8e-06
 9:  2.4087e+00  2.4087e+00  2e-06  1e-06  3e-07  4e-07
10:  2.4087e+00  2.4087e+00  3e-07  1e-07  4e-08  5e-08
11:  2.4087e+00  2.4087e+00  8e-09  4e-09  1e-09  1e-09
Optimal solution found.
1:	1.000000	
w0:	0.450000	
w1:	0.700000	
w0**2:	0.262500	
w0*w1:	0.255000	
w1**2:	0.550000	
w0**3:	0.178125	
w0**2*w1:	0.123750	
w0*w1**2:	0.169500	
w1**3:	0.463000	
w0**4:	0.127321	
w0**3*w1:	0.075799	
w0**2*w1**2:	0.066929	
w0*w1**3:	0.128583	
w1**

## With real samples

Let's try to solve the problem with generated samples.


In [10]:
def compute_expected_moments(xs, alphas):
    moments = {alpha : 0. for alpha in alphas}
    for i, x in enumerate(xs):
        for alpha in alphas:
            moments[alpha] += (monomial(x, alpha) - moments[alpha])/(i+1)
    return moments

def compute_expected_moments_y(ys, xs, alphabs):
    moments = {(alpha, b) : 0. for alpha, b in alphabs}
    for i, (y, x) in enumerate(zip(ys,xs)):
        for alpha, b in alphabs:
            moments[(alpha,b)] += (monomial(x, alpha) * y**b - moments[(alpha,b)])/(i+1)
    return moments

In [11]:
ys, xs = model.sample(1e6)
moments_x = compute_expected_moments(xs, alphas_)
moments_y = compute_expected_moments_y(ys, xs, alphabs)

In [14]:
import MomentMatrix as mm; reload(mm)

constrs = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
display(constrs)
M, sol = mm.solve_moments_with_constraints(R.symbols, constrs, 2, slack=0. 3)
M.pretty_print(sol)
print M.extract_solutions_lasserre(sol['x'], Kmax=2)
print ws

     pcost       dcost       gap    pres   dres   k/t
 0:  1.1121e+00  1.1121e+00  2e+01  4e+00  1e+00  1e+00
 1:  1.6723e+00  1.8479e+00  1e+00  4e-01  2e-01  3e-01
 2:  1.9419e+00  2.0000e+00  4e-01  2e-01  6e-02  1e-01
 3:  2.0087e+00  2.0146e+00  4e-02  2e-02  8e-03  1e-02
 4:  2.0228e+00  2.0240e+00  1e-02  6e-03  2e-03  3e-03
 5:  2.0252e+00  2.0254e+00  2e-03  1e-03  4e-04  5e-04
 6:  2.0258e+00  2.0259e+00  5e-04  2e-04  9e-05  1e-04
 7:  2.0259e+00  2.0260e+00  6e-05  3e-05  1e-05  1e-05
 8:  2.0260e+00  2.0260e+00  1e-05  5e-06  2e-06  2e-06
 9:  2.0260e+00  2.0260e+00  6e-07  3e-07  1e-07  1e-07
10:  2.0260e+00  2.0260e+00  3e-08  1e-08  5e-09  5e-09
Optimal solution found.
1:	1.000000	
w0:	0.449963	
w1:	0.699717	
w0**2:	0.261985	
w0*w1:	0.509486	
w1**2:	0.550088	
w0**3:	0.177585	
w0**2*w1:	0.371110	
w0*w1**2:	0.508481	
w1**3:	0.462652	
w0**4:	-0.128140	
w0**3*w1:	0.205978	
w0**2*w1**2:	0.154082	
w0*w1**3:	0.352857	
w1**4:	0.187953	
the next biggest eigenvalue we are losing 